In [ ]:
# -*- coding: utf-8 -*-
"""
3.4.4 One Figure: Dumbbell plots (5sites vs 170sites) by Biome/Climate

Inputs:
  - ./eval_3_4_2_outputs/all_site_metrics_all_models.csv
  - ./site_metadata.csv  
    (columns: site, biome, climate). 
    If missing, climate is derived from latitude bands and biome='Unknown'.

Outputs:
  - ./eval_3_4_4_outputs/dumbbell_{group}_{metric}_{target}.png
    (group ∈ {biome|climate}, metric ∈ {R2|RMSE}, target ∈ {GPP|NEE})
"""

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# optional: paired t-test
HAVE_SCIPY = True
try:
    from scipy.stats import ttest_rel
except Exception:
    HAVE_SCIPY = False

# ---------- CONFIG ----------
IN_METRICS = Path("./eval_3_4_2_outputs/all_site_metrics_all_models.csv")
IN_MAP     = Path("./site_metadata.csv")  # optional metadata
GROUP_COL  = "biome"                      # change to "climate" for climate-based grouping
METRICS    = ["R2", "RMSE"]               # metrics to visualize
TARGETS    = ["GPP", "NEE"]
SCA_A, SCA_B = "5sites", "170sites"       # scales to compare
OUT_DIR   = Path("./eval_3_4_4_outputs"); OUT_DIR.mkdir(parents=True, exist_ok=True)

# ---------- helpers ----------
def infer_climate_from_lat(lat):
    """Derive climate zone from latitude if metadata is missing."""
    a = abs(float(lat))
    if a < 23.5: return "Tropical"
    if a < 35:   return "Subtropical"
    if a < 55:   return "Temperate"
    if a < 66.5: return "Boreal"
    return "Polar"

def load_data():
    """Load evaluation metrics and merge with site metadata if available."""
    if not IN_METRICS.exists():
        raise FileNotFoundError(IN_METRICS)
    df = pd.read_csv(IN_METRICS)
    # normalize longitude into [-180, 180]
    if (df["lon"] > 180).any():
        df.loc[df["lon"] > 180, "lon"] -= 360.0
    if IN_MAP.exists():
        meta = pd.read_csv(IN_MAP)
        df = df.merge(meta[["site","biome","climate"]], on="site", how="left")
    else:
        # fallback: latitude bands
        print("[INFO] site_metadata.csv not found; using latitude bands as climate, biome='Unknown'.")
        latmap = (df[["site","lat"]].dropna().drop_duplicates("site")
                  .set_index("site")["lat"].to_dict())
        df["climate"] = df["site"].map(lambda s: infer_climate_from_lat(latmap.get(s, np.nan)))
        df["biome"] = "Unknown"
    return df

def paired_table(df, group_col, metric, target):
    """Build paired table of two scales (A, B) for each group."""
    d = df[(df["target"] == target) & df[group_col].notna()].copy()
    A = d[d["scale"]==SCA_A][["site", group_col, metric]].rename(columns={metric:f"{metric}_A"})
    B = d[d["scale"]==SCA_B][["site", group_col, metric]].rename(columns={metric:f"{metric}_B"})
    M = pd.merge(B, A, on=["site", group_col], how="inner")
    if M.empty: return pd.DataFrame()
    rows = []
    for g, G in M.groupby(group_col):
        if len(G)==0: continue
        mean_A = float(np.nanmean(G[f"{metric}_A"]))
        mean_B = float(np.nanmean(G[f"{metric}_B"]))
        delta  = mean_B - mean_A
        pval   = np.nan
        if HAVE_SCIPY and len(G) > 1:
            try:
                pval = float(ttest_rel(G[f"{metric}_B"], G[f"{metric}_A"], nan_policy="omit")[1])
            except Exception:
                pval = np.nan
        rows.append({
            group_col: g, "mean_A": mean_A, "mean_B": mean_B,
            "delta": delta, "n": len(G), "p": pval
        })
    T = pd.DataFrame(rows)
    if T.empty: return T
    # order by performance (R2 descending, RMSE ascending)
    if metric == "R2":
        T = T.sort_values("mean_B", ascending=False)
    else:
        T = T.sort_values("mean_B", ascending=True)
    return T

def star(p):
    """Convert p-value to significance stars."""
    if not np.isfinite(p): return ""
    return "***" if p < 0.001 else ("**" if p < 0.01 else ("*" if p < 0.05 else ""))

def plot_dumbbell(T, group_col, metric, target):
    """Create dumbbell plot showing performance differences between two scales."""
    if T is None or T.empty: return
    ylbl = T[group_col].astype(str).tolist()
    y = np.arange(len(T))[::-1]  # reverse for top-down
    fig, ax = plt.subplots(figsize=(9, max(4.5, 0.35*len(T))), constrained_layout=True)

    # connecting line segments
    for i, yi in enumerate(y):
        ax.plot([T["mean_A"].iat[i], T["mean_B"].iat[i]], [yi, yi],
                color="#c0c0c0", lw=2, zorder=1)
    # markers
    ax.scatter(T["mean_A"], y, color="#1f77b4", s=36, label=SCA_A, zorder=2)
    ax.scatter(T["mean_B"], y, color="#ff7f0e", s=36, label=SCA_B, zorder=3)

    # annotate delta and significance
    for i, yi in enumerate(y):
        d = T["delta"].iat[i]; n = T["n"].iat[i]; p = T["p"].iat[i]
        s = star(p)
        if metric == "R2":
            txt = f"Δ={d:+.3f} {s}  (n={n})"
        else:
            txt = f"Δ={d:+.3f} {s}  (n={n})  {'↓ better' if d<0 else '↑ worse'}"
        x_pos = (T["mean_A"].iat[i] + T["mean_B"].iat[i]) / 2
        ax.text(x_pos, yi+0.15, txt, ha="center", va="bottom", fontsize=8, color="#555555")

    ax.set_yticks(y); ax.set_yticklabels(ylbl)
    ax.set_xlabel(metric)
    ttl = f"{metric} — {SCA_A} vs {SCA_B} by {group_col} ({target})"
    ax.set_title(ttl)
    ax.legend(loc="lower right", frameon=False)
    ax.grid(True, axis="x", ls="--", lw=0.5, alpha=0.4)
    out = OUT_DIR / f"dumbbell_{group_col}_{metric}_{target}.png"
    fig.savefig(out, dpi=240)
    plt.close(fig)
    print(f"[Saved] {out}")

def main():
    df = load_data()
    for target in TARGETS:
        for metric in METRICS:
            T = paired_table(df, GROUP_COL, metric, target)
            if T.empty:
                print(f"[WARN] No overlap for {GROUP_COL}-{metric}-{target}")
                continue
            plot_dumbbell(T, GROUP_COL, metric, target)

if __name__ == "__main__":
    main()


[INFO] site_metadata.csv not found; using latitude bands as climate, biome='Unknown'.
[Saved] eval_3_4_4_outputs/dumbbell_biome_R2_GPP.png
[Saved] eval_3_4_4_outputs/dumbbell_biome_RMSE_GPP.png
[Saved] eval_3_4_4_outputs/dumbbell_biome_R2_NEE.png
[Saved] eval_3_4_4_outputs/dumbbell_biome_RMSE_NEE.png
